# 自动调度

In [1]:
import numpy as np
import tvm
from tvm import te, auto_scheduler

工作负载（workload）实际上是由 TVM DSL 定义的计算 DAG。但是序列化计算 DAG 并有效地匹配它们并不容易。因此，需要使用字符串 `[func_name, [args...]]` 编码 compute dag（返回值为 ``func_name(*args)``）。

这些字符串对于序列化/匹配非常有效，而且不会太长。

当需要 dag 时，解码字符串并调用函数，该函数将返回 dag。

In [2]:
@auto_scheduler.register_workload  # 注意 auto_scheduler 装饰器
def matmul_add(N, L, M, dtype):
    A = te.placeholder((N, L), name="A", dtype=dtype)
    B = te.placeholder((L, M), name="B", dtype=dtype)
    C = te.placeholder((N, M), name="C", dtype=dtype)

    k = te.reduce_axis((0, L), name="k")
    matmul = te.compute(
        (N, M),
        lambda i, j: te.sum(A[i, k] * B[k, j], axis=k),
        name="matmul",
        attrs={"layout_free_placeholders": [B]},  # 启用张量 B 的自动布局转换
    )
    out = te.compute((N, M), lambda i, j: matmul[i, j] + C[i, j], name="out")
    return [A, B, C, out]

## 搜索任务

在定义了函数之后，为 `auto_scheduler` 创建任务来进行搜索。指定这个矩阵乘法的特殊参数：`N=L=M=1024` 和 `dtype="float32"` 创建搜索任务。

In [3]:
# target = tvm.target.Target("llvm")
target = tvm.target.Target("llvm -mcpu=skylake-avx512")

N = L = M = 1024
task = auto_scheduler.SearchTask(func=matmul_add, args=(N, L, M, "float32"), target=target)

# 检查计算图
print("Computational DAG:")
print(task.compute_dag)

Computational DAG:
A = PLACEHOLDER [1024, 1024]
B = PLACEHOLDER [1024, 1024]
matmul(i, j) += (A[i, k]*B[k, j])
C = PLACEHOLDER [1024, 1024]
out(i, j) = (matmul[i, j] + C[i, j])



### 设置参数

In [4]:
log_file = "matmul.json"
tune_option = auto_scheduler.TuningOptions(
    num_measure_trials=10, # 搜索过程中可以使用的测量试验的数量，1000 是很好的搜索收敛值
    measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
    verbose=2,
)

### 运行搜索

In [5]:
# 运行 auto-tuning (search)
task.tune(tune_option)
# 应用最优 schedule
sch, args = task.apply_best(log_file)

/media/pc/data/4tb/lxw/libs/anaconda3/envs/py38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


----------------------------------------------------------------------
------------------------------  [ Search ]
----------------------------------------------------------------------
Generate Sketches		#s: 3
Sample Initial Population	#s: 2014	fail_ct: 2	Time elapsed: 0.58
GA Iter: 0	Max score: 0.9991	Min score: 0.9335	#Pop: 128	#M+: 0	#M-: 0
GA Iter: 4	Max score: 0.9998	Min score: 0.9859	#Pop: 128	#M+: 1384	#M-: 74
EvolutionarySearch		#s: 128	Time elapsed: 2.40
----------------------------------------------------------------------
------------------------------  [ Measure ]
----------------------------------------------------------------------
Get 10 programs to measure:
..........*E*E*E*E***E*E*E*E
No: 1	GFLOPS: 0.00 / 0.00	results: MeasureResult(error_type:RuntimeDeviceError, error_msg:Traceback (most recent call last):
  File "/media/pc/data/4tb/lxw/libs/anaconda3/envs/py38/lib/python3.8/site-packages/tvm/auto_scheduler/utils.py", line 295, in call_func_with_timeout
    res = work

In [7]:
print("Lowered TIR:")
mod = tvm.lower(sch, args, simple_mode=True)
mod.show()

Lowered TIR:
@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[1048576, "float32"], B: T.Buffer[1048576, "float32"], C: T.Buffer[1048576, "float32"], out: T.Buffer[1048576, "float32"]) -> None:
        # function attr dict
        T.func_attr({"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True})
        T.preflattened_buffer(A, [1024, 1024], dtype="float32", data=A.data)
        T.preflattened_buffer(B, [1024, 1024], dtype="float32", data=B.data)
        T.preflattened_buffer(C, [1024, 1024], dtype="float32", data=C.data)
        T.preflattened_buffer(out, [1024, 1024], dtype="float32", data=out.data)
        # body
        auto_scheduler_layout_transform = T.allocate([1048576], "float32", "global")
        matmul = T.allocate([1048576], "float32", "global")
        for ax0_ax1_fused_ax2_fused in T.parallel(131072):
            for ax3, ax4, ax5 in T.grid(2, 2, 2):
                auto_scheduler_layout_transform[ax0_ax1_fused_ax2_f